In [43]:
import os
import config as c
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.datasets import fetch_california_housing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go

from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix
from utils.metrics import NIC
from utils.dataset import *
from utils.metrics import NIC
from sklearn.metrics import mean_squared_error
from utils.model import Model, get_combine_predictions

init_notebook_mode(connected=True)

warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [177]:
df_funnel, df_appl, df_aum, df_balance, df_client, df_com, df_deals, df_payments = get_dataset(True)

In [7]:
df_trxn = load_trxn()

# Создание фич

In [26]:
df_funnel = get_mean_month_costs(df_funnel, df_trxn)

In [27]:
df_funnel

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,region_cd,month_payments,freq_client_segment,freq_feature_1,freq_feature_9,feature_1_client_segment,freq_feature_1_client_segment,gender,age,region,city,citizenship,education,job_type,mean_month_costs
0,-9221941791080978530,0,NaN,1,9,13,1821184.0,79089.0,132280.0,0.0,166263.0,1349.512505,1596130.0,194,10.771533,16,NaN,3865,2012,476,9_13,334,55138076,35,22,688,11175746,26418121,41838474,NaN
1,-9220369594510368140,0,NaN,1,1,13,3642369.0,11288.0,6556.0,0.0,12464.0,713.244699,420428.0,119,9.331761,50,NaN,3865,2491,1228,1_13,461,52748146,54,8,729,11175746,26418121,41838474,NaN
2,-9220236243053692422,1,32920.9,2,1,3,3642369.0,24469.0,23073.0,16764.0,49060.0,1068.305200,436482.0,89,10.105407,36,15737.333333,4139,2491,2488,1_3,351,52748146,39,52,953,11175746,26418121,41838474,27723.000000
3,-9220233431709087652,0,NaN,1,6,1,1371274.0,77447.0,27857.0,61894.0,15563.0,1170.788196,1177470.0,119,10.322954,86,32787.000000,2221,2388,1228,6_1,149,52748146,27,0,41,11175746,64282531,41838474,106162.200000
4,-9219699286371310531,0,NaN,1,7,3,352826.0,5500.0,0.0,4244.0,0.0,515.648136,204534.0,89,8.614138,62,4629.636364,4139,2675,2488,7_3,461,55138076,60,60,4166,11175746,26418121,41838474,7510.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21493,9219024469308275500,0,NaN,1,4,3,6070615.0,10658.0,14642.0,12445.0,30515.0,730.068490,198936.0,74,9.274441,68,7467.200000,4139,2516,739,4_3,537,52748146,49,48,714,11175746,26418121,41838474,18271.875000
21494,9219416956046769492,0,NaN,1,7,14,536675.0,1871.0,13483.0,0.0,26355.0,297.709590,68178.0,99,7.539027,78,NaN,4689,2675,1971,7_14,588,55138076,34,25,33,11175746,26418121,41838474,NaN
21495,9219968212912398941,0,NaN,1,6,13,306824.0,8182.0,0.0,0.0,5370.0,554.231901,253862.0,149,8.894670,86,NaN,3865,2388,1181,6_13,460,55138076,44,0,2,11175746,26418121,41838474,NaN
21496,9220335314469087849,0,NaN,1,5,3,615909.0,18654.0,0.0,25541.0,0.0,784.576319,486293.0,89,9.453208,28,15792.523810,4139,2637,2488,5_3,474,52748146,33,32,53,11175746,26418121,41838474,38589.200000


In [176]:
df_payments.pmnts_name

134255    0
49583     0
120137    0
8086      0
26053     0
         ..
115806    0
66369     0
38074     0
84043     0
98439     0
Name: pmnts_name, Length: 188068, dtype: int32

In [206]:
df_funnel = get_pensioner(df_funnel, df_payments)

In [207]:
df = df_funnel.copy()

In [208]:
x = df[df['sale_amount'].fillna(0) > 0].drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()
y = df[df['sale_amount'].fillna(0) > 0]['sale_amount'].to_numpy().reshape(-1, 1)

model_sale_amount = Model('model_sale_amount')
model_sale_amount.fit(x, y)
print(mean_squared_error(y, model_sale_amount.predict(x)))


x = df.drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()
y = df['sale_flg'].to_numpy().reshape(-1, 1)

model_sale_flg = Model('model_sale_flg')
model_sale_flg.fit(x, y)
print(mean_squared_error(y, model_sale_flg.predict(x)))

x = df.drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()

target = get_combine_predictions(x, model_sale_flg, model_sale_amount)
print('NIC:', NIC(target, df['sale_amount'], df['contacts']))

10it [00:10,  1.01s/it]
100%|██████████| 10/10 [00:00<00:00, 100.00it/s]
0it [00:00, ?it/s]

Models score: -440.5667619479609
1070472532.4718974


10it [00:10,  1.07s/it]
 20%|██        | 2/10 [00:00<00:00, 19.23it/s]

Models score: -2545.6529688549567


 20%|██        | 2/10 [00:00<00:00, 18.35it/s]

0.1004376184597435


100%|██████████| 10/10 [00:00<00:00, 18.45it/s]

NIC: 4708.221888082613


In [450]:
df_trxnl['year_month'] = df_trxnl.tran_time.dt.month + df_trxnl.tran_time.dt.year * 10

month_costs = df_trxnl.groupby(['client_id', 'year_month']).sum()['tran_amt_rur'].reset_index()
month_costs = month_costs.rename(columns={'tran_amt_rur': 'mean_month_costs'})
month_costs = month_costs.groupby('client_id').mean()['mean_month_costs'].reset_index()
df_funnel = pd.concat([df_funnel.set_index('client_id'), month_costs.set_index('client_id')], axis=1).reset_index()

In [455]:
month_payments = df_payments.groupby('client_id').mean()['sum_rur'].reset_index()
month_payments.rename(columns={'sum_rur': 'month_payments'})

df_funnel = pd.concat([df_funnel.set_index('client_id'), month_payments.set_index('client_id')], axis=1).reset_index()

In [401]:
df_funnel['delta'] = df_funnel.sum_rur - df_funnel.tran_amt_rur

In [402]:
df_funnel.corr()

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,...,feature_8,feature_9,feature_10,region_cd,age,region,city,tran_amt_rur,sum_rur,delta
client_id,1.000000,0.002857,0.024852,0.001679,-0.002944,-0.002263,0.006095,0.014837,0.002039,0.006921,...,0.008229,-0.002983,0.010589,-0.006946,-0.007511,-0.007042,0.000983,0.011113,0.012488,-0.018530
sale_flg,0.002857,1.000000,NaN,0.026489,-0.038500,-0.301903,0.061878,0.023095,0.080969,0.124936,...,0.102033,-0.169604,0.133996,0.031090,-0.023028,-0.056472,-0.098920,0.045029,0.018338,-0.046271
sale_amount,0.024852,NaN,1.000000,-0.014242,-0.048497,-0.054515,0.153004,0.355598,0.443969,0.180681,...,0.326322,-0.153427,0.421133,0.169763,0.094231,-0.146293,-0.104871,0.244278,0.214024,-0.230047
contacts,0.001679,0.026489,-0.014242,1.000000,0.000061,-0.127788,0.056231,0.009877,0.043545,0.023176,...,0.044323,-0.044366,0.053606,0.025433,-0.023065,-0.035442,-0.017541,-0.016523,-0.022700,0.012960
feature_1,-0.002944,-0.038500,-0.048497,0.000061,1.000000,-0.021809,-0.076170,0.007475,0.022636,0.040125,...,0.020056,0.364472,0.034297,0.000467,-0.023780,-0.003665,0.003439,0.029223,0.040027,-0.022731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
region,-0.007042,-0.056472,-0.146293,-0.035442,-0.003665,0.086111,-0.087676,-0.023066,-0.027156,-0.115405,...,-0.150888,0.093235,-0.150904,-0.379073,-0.006608,1.000000,0.216318,-0.044136,-0.098197,0.039379
city,0.000983,-0.098920,-0.104871,-0.017541,0.003439,0.040424,-0.057365,-0.041689,-0.041988,-0.057118,...,-0.109249,0.059237,-0.110128,-0.227713,-0.015504,0.216318,1.000000,-0.051339,-0.067467,0.051552
tran_amt_rur,0.011113,0.045029,0.244278,-0.016523,0.029223,-0.042389,0.059634,0.147651,0.123501,0.244396,...,0.210783,-0.012078,0.223511,0.039404,-0.039863,-0.044136,-0.051339,1.000000,0.245065,-0.982651
sum_rur,0.012488,0.018338,0.214024,-0.022700,0.040027,-0.067893,0.408856,0.776815,0.203548,0.889357,...,0.387621,-0.075298,0.394329,0.074781,-0.014582,-0.098197,-0.067467,0.245065,1.000000,-0.061003


In [446]:
features

Index(['feature_1', 'client_segment', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'region_cd', 'age', 'tran_amt_rur', 'sum_rur', 'delta'],
      dtype='object')

In [444]:
features = df_funnel.columns[4:]

In [445]:
features = features.drop(['region', 'gender', 'city', 'citizenship', 'education', 'job_type'])

In [437]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostClassifier(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')
model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)
model.score(x_test, y_test)

0.8021474385530034

In [442]:
features = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'client_segment', 'region_cd']

In [443]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostClassifier(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')
model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)
model.score(x_test, y_test)

0.8005380704638347

In [416]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)

score = 0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

    model = CatBoostRegressor(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')

    model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

    score += NIC(model.predict(x), df_funnel.sale_amount, df_funnel.contacts, 0.55)
    
print('==||=='*10)
print(score / 10)

==||====||====||====||====||====||====||====||====||====||==
87.83901737000001


In [417]:
features = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'client_segment', 'region_cd']

In [418]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)

score = 0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

    model = CatBoostRegressor(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')

    model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

    score += NIC(model.predict(x), df_funnel.sale_amount, df_funnel.contacts, 0.5)
    
print('==||=='*10)
print(score / 10)

==||====||====||====||====||====||====||====||====||====||==
87.89666834


In [462]:
kf = KFold(n_splits=10)
for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [403]:
features = df_funnel.columns[4:]

In [464]:
np.mean(df_funnel.contacts)

1.2265792166713183

In [302]:
from sklearn.metrics import mean_absolute_error

In [303]:
x = df_funnel[df_funnel.sale_amount.fillna(0) != 0][features]
y = df_funnel[df_funnel.sale_amount.fillna(0) != 0].sale_amount.to_numpy().reshape(-1, 1)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostRegressor(iterations=100,
                          learning_rate=3e-2,
                          l2_leaf_reg=3.0,  # any pos value
                          depth=5,  # int up to 16
                          min_data_in_leaf=1,  # 1,2,3,4,5
                          rsm=1,  # 0.01 .. 1.0
                          langevin=False,
                          task_type="GPU",
                          devices='0:1')

model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

print(model.predict(x) - y)

[[  -2889.54100897   40174.89651451   -1725.83336481 ...   35184.75600508
    31302.83140168   36645.46463547]
 [ -35810.49100897    7253.94651451  -34646.78336481 ...    2263.80600508
    -1618.11859832    3724.51463547]
 [  21753.20899103   64817.64651451   22916.91663519 ...   59827.50600508
    55945.58140168   61288.21463547]
 ...
 [-126821.19100897  -83756.75348549 -125657.48336481 ...  -88746.89399492
   -92628.81859832  -87286.18536453]
 [  -9473.69100897   33590.74651451   -8309.98336481 ...   28600.60600508
    24718.68140168   30061.31463547]
 [  -9772.79100897   33291.64651451   -8609.08336481 ...   28301.50600508
    24419.58140168   29762.21463547]]


In [314]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(x.fillna(0), y)

In [ ]:
model.score(x_test, y_test)

In [317]:
mean_absolute_error(y, model.predict(x.fillna(0)))

24748.28564380754

In [318]:
mean_absolute_error(y, model.predict(x.fillna(0))*0+25000)

30057.40585393578